In [10]:
import pandas as pd
import re
import numpy as np

# Read the Excel file
df = pd.read_excel('Duties Import Jan 99.xlsx')

# Extract first 4 digits from Goods code
# Convert to string first to handle any numeric values, then take first 4 characters
df['Goods_Code_4'] = df['Goods code'].astype(str).str[:4]

df = df[df['Origin'] == 'ERGA OMNES']

# --- Added: clean/parse Duty column to numeric duty rate (percent as float) ---
def parse_duty_rate(val):
    """
    Parse duty strings and return duty rate as float (e.g. '12.000 %' -> 0.12).
    Returns np.nan when no percentage can be found (e.g. 'NAR', 'Cond: ...').
    """
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    # look for percentage like '12.000 %' or '12%' with optional spaces and commas
    m = re.search(r'([\d]+[.,]?\d*)\s*%', s)
    if m:
        num = m.group(1).replace(',', '.')
        try:
            return float(num) / 100.0
        except ValueError:
            return np.nan
    # fallback: if string is purely numeric (no percent sign) take it as percent*
    m2 = re.search(r'^([\d]+[.,]?\d*)$', s)
    if m2:
        num = m2.group(1).replace(',', '.')
        try:
            return float(num) / 100.0
        except ValueError:
            return np.nan
    # no quantitative value found
    return np.nan

# create new column with parsed duty rate
df['Duty_rate'] = df['Duty'].apply(parse_duty_rate)

# summary of parsing
total_rows = len(df)
parsed_count = df['Duty_rate'].notna().sum()
unparsed_count = total_rows - parsed_count
print(f"Duty parsing: {parsed_count}/{total_rows} parsed, {unparsed_count} unparsed (set as NaN).")

# --- NEW: pick the biggest (max) duty rate per 4-digit goods code and keep only code + that rate ---
max_duty = df.groupby('Goods_Code_4', as_index=False)['Duty_rate'].max()

# remove codes without a numeric duty rate (max == NaN)
max_duty = max_duty.dropna(subset=['Duty_rate'])

# optional: sort by duty rate descending for quick inspection
max_duty = max_duty.sort_values('Duty_rate', ascending=False).reset_index(drop=True)

print(f"Numeric max duty found for {len(max_duty)} codes. Sample:\n", max_duty.head())

# --- Save the code + max duty rate to an Excel file ---
output_file = 'Duties_Import_Jan_99_max_duty.xlsx'
try:
    max_duty.to_excel(output_file, index=False)
    print(f"Max duty per code saved to: {output_file}")
except Exception as e:
    print("Failed to save max duty to Excel:", e)


/Users/admin/CS_Projects/YandexDS/.venv1/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Duty parsing: 14189/22856 parsed, 8667 unparsed (set as NaN).
Numeric max duty found for 1349 codes. Sample:
   Goods_Code_4  Duty_rate
0         8305      0.788
1         2403      0.749
2         8431      0.708
3         8427      0.708
4         7019      0.690
Max duty per code saved to: Duties_Import_Jan_99_max_duty.xlsx
